##Initialization

In [1]:
import os
os.getcwd()

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
new_path = '/content/drive/My Drive/IBM Mangrove Project/Data/India and Indonesia'
os.chdir(new_path)

Mounted at /content/drive


In [3]:
# Verify the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content/drive/My Drive/IBM Mangrove Project/Data/India and Indonesia


## Importing Libraries

In [4]:

# Import necessary libraries
import numpy as np
import pandas as pd
import re
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from itertools import product
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Install necessary libraries (if not already available in Colab)
!pip install openpyxl  # If using Excel files that require openpyxl

## Inspecting and optimizing the structure of the Master Data

In [6]:
master_data = pd.read_excel('/content/drive/My Drive/IBM Mangrove Project/India and Indonesia dataADB.xlsx')
print(master_data.shape)
master_data.head()

(102, 32)


,Economy,Indicator,Unit of Measure,2000,2001,2002,2003,2004,2005,2006,...,2019,2020,2021,2022,Definition,Data Coverage,Calendar Year,Base Year,Source,Footnotes
0,Indonesia,GDP at current prices,Indonesian Rupiah,1389769850000000,1646322100000000,1821833360000000,2013674600000000,2295826200000000,2774281100000000,3339216800000000,...,15832657200000000,15438017500000000,16970789200000000,...,Unduplicated market value of the total product...,From 2000 to 2022,Calendar Year,NaN,BPS Statistics Indonesia,NaN
1,Indonesia,Agriculture (% of GDP),percent of GDP,15.60197,15.29026,15.45645,15.18535,14.33578,13.12662,12.9738,...,13.258161,14.221554,13.840219,...,Value-added of the agricultural sector as perc...,From 2000 to 2022,Calendar Year,NaN,BPS Statistics Indonesia,NaN
2,Indonesia,Industry (% of GDP),percent of GDP,45.9254,46.45484,44.46292,43.74957,44.62762,46.54106,46.94356,...,40.623372,39.700684,41.538108,...,Value-added of the industry sector as percent ...,From 2000 to 2022,Calendar Year,NaN,BPS Statistics Indonesia,NaN
3,Indonesia,Per capita GDP,Indonesian Rupiah,6737801.11,7890613.68,8616358.48,9398657.56,10576110.92,12618857.69,14991080.91,...,59060096.61,56938722.67,62236558.33535,...,"GDP at current prices, divided by the midyear ...",From 2000 to 2022,Calendar Year,NaN,2000–2020: BPS Statistics Indonesia. 2021: Asi...,NaN
4,Indonesia,"Road Indicators Network, Total (km)",kilometer,348083,352762,357026,357959,372928,391008,406569,...,544474,548366,...,...,This includes both paved and unpaved roads. Pa...,From 2000 to 2022,NaN,NaN,Asian Development Bank,NaN


In [7]:
# Identify all year columns from 2000 to 2022, taking into account that they are integers
year_columns = [col for col in master_data.columns if isinstance(col, int) and 2000 <= col <= 2022]

# Now we build the list of columns to keep, which includes 'Economy', 'Indicator', and the year columns
columns_to_keep = ['Economy', 'Indicator'] + year_columns

# Select only the necessary columns
master_data = master_data[columns_to_keep]

# Remove the 'Unit of Measure' column if it exists
if 'Unit of Measure' in master_data.columns:
    master_data.drop('Unit of Measure', axis=1, inplace=True)

# Convert all column names to strings
master_data.columns = master_data.columns.map(str)

In [8]:
print(master_data.shape)
master_data.head()

(102, 25)


,Economy,Indicator,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Indonesia,GDP at current prices,1389769850000000,1646322100000000,1821833360000000,2013674600000000,2295826200000000,2774281100000000,3339216800000000,3950893200000000,...,9546134000000000,10569705300000000,11526332800000000,12401728500000000,13589825700000000,14838756000000000,15832657200000000,15438017500000000,16970789200000000,...
1,Indonesia,Agriculture (% of GDP),15.60197,15.29026,15.45645,15.18535,14.33578,13.12662,12.9738,13.71668,...,13.697926,13.677701,13.931545,13.977908,13.683466,13.350102,13.258161,14.221554,13.840219,...
2,Indonesia,Industry (% of GDP),45.9254,46.45484,44.46292,43.74957,44.62762,46.54106,46.94356,46.79914,...,43.724902,43.000216,41.350576,40.764156,40.956027,41.409699,40.623372,39.700684,41.538108,...
3,Indonesia,Per capita GDP,6737801.11,7890613.68,8616358.48,9398657.56,10576110.92,12618857.69,14991080.91,17509564.71,...,38365914.7,41915863.36,45119612.06,47937722.5,51891177.41,55992136.29,59060096.61,56938722.67,62236558.33535,...
4,Indonesia,"Road Indicators Network, Total (km)",348083,352762,357026,357959,372928,391008,406569,421535,...,508000,517753,529073,537838,539353,542310,544474,548366,...,...


In [9]:
# Convert year columns to float, setting errors='coerce' to turn non-convertible values to NaN
for col in master_data.columns[2:]:  # Assuming the first two columns are 'Economy' and 'Indicator'
    master_data[col] = pd.to_numeric(master_data[col], errors='coerce')

# Verify the changes
print(master_data.dtypes)

Economy       object
Indicator     object
2000         float64
2001         float64
2002         float64
2003         float64
2004         float64
2005         float64
2006         float64
2007         float64
2008         float64
2009         float64
2010         float64
2011         float64
2012         float64
2013         float64
2014         float64
2015         float64
2016         float64
2017         float64
2018         float64
2019         float64
2020         float64
2021         float64
2022         float64
dtype: object


## Importing and Transforming New Data

### Governance Indicators

In [10]:
governance_indicators_df = pd.read_csv('/content/drive/My Drive/IBM Mangrove Project/Data/India and Indonesia/Worldwide Governance Indicators - India and Indonesia.csv')
print(governance_indicators_df.shape)
governance_indicators_df.head()

(55, 28)


,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,India,IND,Control of Corruption: Estimate,CC.EST,-0.381090,-0.258727,-0.403302,-0.555382,-0.456321,-0.448476,...,-0.517086,-0.457155,-0.406171,-0.336942,-0.291452,-0.229412,-0.302205,-0.292916,-0.316494,-0.321811
1,India,IND,Control of Corruption: Number of Sources,CC.NO.SRC,7.000000,7.000000,7.000000,7.000000,8.000000,10.000000,...,14.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,12.000000,12.000000,12.000000
2,India,IND,Control of Corruption: Percentile Rank,CC.PER.RNK,43.010754,49.732620,43.617020,36.507938,40.740742,37.931034,...,36.966824,39.903847,42.857143,43.809525,45.714287,47.619049,44.761906,45.238094,45.238094,44.339622
3,India,IND,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,29.569893,35.828876,32.446808,25.396826,29.629629,29.064039,...,26.540285,29.807692,30.952381,33.809525,39.047619,39.523811,33.809525,32.857143,31.904762,32.547169
4,India,IND,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,53.763439,54.545456,52.659573,46.031746,50.793652,49.753696,...,46.919430,50.000000,51.428570,53.333332,53.333332,55.714287,52.857143,52.857143,50.476189,51.886791


We need to perform transformation on the data and ensure it has the same structure and data type as the master data so that we can perform concatenation later.

In [11]:
# Rename 'Series Name' column to 'Indicator'
governance_indicators_df.rename(columns={'Series Name': 'Indicator'}, inplace=True)

# Use regex to identify all year columns between 1996 to 2022
year_columns = [col for col in governance_indicators_df.columns if re.search(r'\d{4}', col)]

# Rename year columns by removing any extra characters and keeping only the year
governance_indicators_df.columns = [re.sub(r'.*\[YR(\d{4})\]', r'\1', col) if col in year_columns else col for col in governance_indicators_df.columns]

# Select only the columns from the year 2000 to 2022 and other necessary columns
columns_to_keep = ['Country Name', 'Indicator'] + [str(year) for year in range(2000, 2023) if str(year) in governance_indicators_df.columns]
governance_indicators_df = governance_indicators_df[columns_to_keep]

# Drop the 'Country Code' and 'Series Code' columns if they exist
for column in ['Country Code', 'Series Code']:
    if column in governance_indicators_df.columns:
        governance_indicators_df.drop(column, axis=1, inplace=True)

In [12]:
print(governance_indicators_df.shape)
governance_indicators_df.head()

(55, 24)


,Country Name,Indicator,2000,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,India,Control of Corruption: Estimate,-0.403302,-0.555382,-0.456321,-0.448476,-0.363161,-0.274563,-0.397690,-0.339098,...,-0.517086,-0.457155,-0.406171,-0.336942,-0.291452,-0.229412,-0.302205,-0.292916,-0.316494,-0.321811
1,India,Control of Corruption: Number of Sources,7.000000,7.000000,8.000000,10.000000,11.000000,14.000000,14.000000,14.000000,...,14.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,12.000000,12.000000,12.000000
2,India,Control of Corruption: Percentile Rank,43.617020,36.507938,40.740742,37.931034,44.878048,47.317074,41.262135,43.689320,...,36.966824,39.903847,42.857143,43.809525,45.714287,47.619049,44.761906,45.238094,45.238094,44.339622
3,India,"Control of Corruption: Percentile Rank, Lower ...",32.446808,25.396826,29.629629,29.064039,34.146343,37.560974,31.553398,33.980583,...,26.540285,29.807692,30.952381,33.809525,39.047619,39.523811,33.809525,32.857143,31.904762,32.547169
4,India,"Control of Corruption: Percentile Rank, Upper ...",52.659573,46.031746,50.793652,49.753696,52.195122,55.121952,52.427185,53.398060,...,46.919430,50.000000,51.428570,53.333332,53.333332,55.714287,52.857143,52.857143,50.476189,51.886791


In [13]:
# Rename 'Country Name' to 'Economy'
governance_indicators_df.rename(columns={'Country Name': 'Economy'}, inplace=True)

# Convert all column names to strings
governance_indicators_df.columns = governance_indicators_df.columns.map(str)

# Convert year column data to float64
for col in governance_indicators_df.columns[2:]:  # Assuming the first two columns are 'Economy' and 'Indicator'
    governance_indicators_df[col] = pd.to_numeric(governance_indicators_df[col], errors='coerce')

# Add missing year columns from 2000 to 2022 and fill them with NaN
all_years = [str(year) for year in range(2000, 2023)]
missing_years = set(all_years) - set(governance_indicators_df.columns)
for year in missing_years:
    governance_indicators_df[year] = np.nan

# Reorder the columns to match the order in master_data
governance_indicators_df = governance_indicators_df[['Economy', 'Indicator'] + all_years]

# Verify the structure and data types
print(governance_indicators_df.head())
print(governance_indicators_df.dtypes)

  Economy                                          Indicator       2000  2001  \
0   India                    Control of Corruption: Estimate  -0.403302   NaN   
1   India           Control of Corruption: Number of Sources   7.000000   NaN   
2   India             Control of Corruption: Percentile Rank  43.617020   NaN   
3   India  Control of Corruption: Percentile Rank, Lower ...  32.446808   NaN   
4   India  Control of Corruption: Percentile Rank, Upper ...  52.659573   NaN   

        2002       2003       2004       2005       2006       2007  ...  \
0  -0.555382  -0.456321  -0.448476  -0.363161  -0.274563  -0.397690  ...   
1   7.000000   8.000000  10.000000  11.000000  14.000000  14.000000  ...   
2  36.507938  40.740742  37.931034  44.878048  47.317074  41.262135  ...   
3  25.396826  29.629629  29.064039  34.146343  37.560974  31.553398  ...   
4  46.031746  50.793652  49.753696  52.195122  55.121952  52.427185  ...   

        2013       2014       2015       2016       2017

In [14]:
master_data

,Economy,Indicator,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Indonesia,GDP at current prices,1.389770e+15,1.646322e+15,1.821833e+15,2.013675e+15,2.295826e+15,2.774281e+15,3.339217e+15,3.950893e+15,...,9.546134e+15,1.056971e+16,1.152633e+16,1.240173e+16,1.358983e+16,1.483876e+16,1.583266e+16,1.543802e+16,1.697079e+16,NaN
1,Indonesia,Agriculture (% of GDP),1.560197e+01,1.529026e+01,1.545645e+01,1.518535e+01,1.433578e+01,1.312662e+01,1.297380e+01,1.371668e+01,...,1.369793e+01,1.367770e+01,1.393154e+01,1.397791e+01,1.368347e+01,1.335010e+01,1.325816e+01,1.422155e+01,1.384022e+01,NaN
2,Indonesia,Industry (% of GDP),4.592540e+01,4.645484e+01,4.446292e+01,4.374957e+01,4.462762e+01,4.654106e+01,4.694356e+01,4.679914e+01,...,4.372490e+01,4.300022e+01,4.135058e+01,4.076416e+01,4.095603e+01,4.140970e+01,4.062337e+01,3.970068e+01,4.153811e+01,NaN
3,Indonesia,Per capita GDP,6.737801e+06,7.890614e+06,8.616358e+06,9.398658e+06,1.057611e+07,1.261886e+07,1.499108e+07,1.750956e+07,...,3.836591e+07,4.191586e+07,4.511961e+07,4.793772e+07,5.189118e+07,5.599214e+07,5.906010e+07,5.693872e+07,6.223656e+07,NaN
4,Indonesia,"Road Indicators Network, Total (km)",3.480830e+05,3.527620e+05,3.570260e+05,3.579590e+05,3.729280e+05,3.910080e+05,4.065690e+05,4.215350e+05,...,5.080000e+05,5.177530e+05,5.290730e+05,5.378380e+05,5.393530e+05,5.423100e+05,5.444740e+05,5.483660e+05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,India,Direct Economic Loss Attributed to Disasters (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN
98,India,Proportion of Urban Population Living in Slums...,5.529477e+01,NaN,5.459656e+01,NaN,5.389834e+01,NaN,5.320013e+01,NaN,...,NaN,5.040728e+01,NaN,4.970906e+01,NaN,4.901085e+01,NaN,4.901085e+01,NaN,NaN
99,India,Coverage of Protected Areas in Relation to Mar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.570000e-03,NaN
100,India,Protected Marine Areas (Exclusive Economic Zon...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.281073e+02,NaN


In [15]:
def concat_country_specific_data(master_data, new_data, countries):
    """
    Concatenates new data into the master data, ensuring that data for each country in 'countries' list
    is inserted into the correct location in the master data.

    Parameters:
    - master_data: DataFrame containing the master data.
    - new_data: DataFrame containing the new data to be added.
    - countries: List of country names as strings, indicating the order in which the countries appear in the master data.

    Returns:
    - A new DataFrame with the new data concatenated into the master data.
    """
    final_data_parts = []

    # Previous index where we should start the next data insertion
    prev_index = 0

    for country in countries:
        # Filter new data for the current country
        country_new_data = new_data[new_data['Economy'] == country]

        # Find the index range for the current country in the master data
        country_master_data = master_data[master_data['Economy'] == country]
        start_index = country_master_data.index.min()
        end_index = country_master_data.index.max() + 1  # +1 to include the last index in the slice

        # Concatenate the part of master data up to the current country's data with the new country data
        if not country_new_data.empty:
            combined_country_data = pd.concat([master_data.iloc[prev_index:start_index], country_new_data, master_data.iloc[start_index:end_index]], ignore_index=True)
        else:
            combined_country_data = master_data.iloc[prev_index:end_index]

        final_data_parts.append(combined_country_data)

        # Update the previous index for the next iteration
        prev_index = end_index

    # Concatenate any remaining data from master_data after the last country
    if prev_index < len(master_data):
        final_data_parts.append(master_data.iloc[prev_index:])

    # Combine all parts back into a single DataFrame
    final_combined_data = pd.concat(final_data_parts, ignore_index=True)

    return final_combined_data

In [16]:
# Countries for which the data is being added
countries = ['Indonesia', 'India']

# Use the function to concatenate governance_indicators_df into master_data
master_data_1 = concat_country_specific_data(master_data, governance_indicators_df, countries)

# Verify the combined data
print(master_data_1)

       Economy                                          Indicator       2000  \
0    Indonesia                    Control of Corruption: Estimate  -0.908694   
1    Indonesia           Control of Corruption: Number of Sources   7.000000   
2    Indonesia             Control of Corruption: Percentile Rank  21.808510   
3    Indonesia  Control of Corruption: Percentile Rank, Lower ...   8.510638   
4    Indonesia  Control of Corruption: Percentile Rank, Upper ...  34.042553   
..         ...                                                ...        ...   
147      India  Direct Economic Loss Attributed to Disasters (...        NaN   
148      India  Proportion of Urban Population Living in Slums...  55.294770   
149      India  Coverage of Protected Areas in Relation to Mar...        NaN   
150      India  Protected Marine Areas (Exclusive Economic Zon...        NaN   
151      India             Number of Persons Affected by Disaster        NaN   

     2001       2002       2003       2

### Economic Fitness

In [17]:
economic_fitness_df = pd.read_csv('/content/drive/My Drive/IBM Mangrove Project/Data/India and Indonesia/Economic_Fitness India and Indonesia.csv')
print(economic_fitness_df.shape)
economic_fitness_df.head()

(8, 25)


,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015]
0,India,IND,Economic Fitness Metric (Legacy),EF.EFM.OVRL.XD,2.6222,2.6760,3.1265,3.1755,3.2775,3.5839,...,3.7963,3.5818,3.8155,3.9348,3.6693,3.7188,4.0268,4.1436,4.3043,4.395582
1,India,IND,"Economic Fitness Ranking (1 = high, 149 = low)...",EF.EFM.RANK.XD,16.0000,17.0000,15.0000,15.0000,14.0000,14.0000,...,11.0000,12.0000,10.0000,10.0000,10.0000,10.0000,8.0000,9.0000,8.0000,7.000000
2,Indonesia,IDN,Economic Fitness Metric (Legacy),EF.EFM.OVRL.XD,1.0307,1.1111,1.1514,1.3051,1.4004,1.5487,...,1.6557,1.5796,1.6062,1.7914,1.4658,1.4986,1.4925,1.6352,1.5812,1.564621
3,Indonesia,IDN,"Economic Fitness Ranking (1 = high, 149 = low)...",EF.EFM.RANK.XD,40.0000,41.0000,39.0000,34.0000,34.0000,31.0000,...,27.0000,33.0000,32.0000,25.0000,36.0000,37.0000,35.0000,33.0000,36.0000,32.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Rename necessary columns to match master_data structure
economic_fitness_df.rename(columns={'Country Name': 'Economy', 'Series Name': 'Indicator'}, inplace=True)

# Drop columns that are not needed
economic_fitness_df.drop(['Country Code', 'Series Code'], axis=1, inplace=True)

# Convert all column names to strings
economic_fitness_df.columns = economic_fitness_df.columns.map(str)

# Initialize a new dictionary to hold the updated column names
new_columns = {}

# Iterate over each column, extract the year, and update the column name if it's a year
for col in economic_fitness_df.columns:
    match = re.search(r'(\d{4})', col)
    if match:
        year = match.group(1)
        new_columns[col] = year
    else:
        new_columns[col] = col

# Update the column names with the new names
economic_fitness_df.rename(columns=new_columns, inplace=True)

# Convert year column data to float64
for col in economic_fitness_df.columns[2:]:  # Assuming the first two columns are 'Economy' and 'Indicator'
    economic_fitness_df[col] = pd.to_numeric(economic_fitness_df[col], errors='coerce')

# Add missing year columns from 2000 to 2022 and fill them with NaN
all_years = [str(year) for year in range(2000, 2023)]
missing_years = set(all_years) - set(economic_fitness_df.columns[2:])
for year in missing_years:
    economic_fitness_df[year] = np.nan

# Reorder the columns to match the order in master_data
economic_fitness_df = economic_fitness_df[['Economy', 'Indicator'] + all_years]

# Remove rows where all columns except 'Economy' and 'Indicator' are NaN
economic_fitness_df.dropna(how='all', subset=economic_fitness_df.columns[2:], inplace=True)

# Verify the structure and data types
print(economic_fitness_df.head())
print(economic_fitness_df.dtypes)

     Economy                                          Indicator     2000  \
0      India                   Economic Fitness Metric (Legacy)   3.5839   
1      India  Economic Fitness Ranking (1 = high, 149 = low)...  14.0000   
2  Indonesia                   Economic Fitness Metric (Legacy)   1.5487   
3  Indonesia  Economic Fitness Ranking (1 = high, 149 = low)...  31.0000   

      2001     2002     2003     2004     2005     2006     2007  ...  \
0   3.8693   3.9160   3.7727   4.0137   4.0312   3.7963   3.5818  ...   
1  12.0000  12.0000  11.0000  10.0000  10.0000  11.0000  12.0000  ...   
2   1.5931   1.5922   1.5952   1.6356   1.4912   1.6557   1.5796  ...   
3  28.0000  29.0000  29.0000  29.0000  32.0000  27.0000  33.0000  ...   

      2013     2014       2015  2016  2017  2018  2019  2020  2021  2022  
0   4.1436   4.3043   4.395582   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1   9.0000   8.0000   7.000000   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2   1.6352   1.5812   1.5646

In [19]:
# Countries for which the data is being added
countries = ['Indonesia', 'India']

# Use the function to concatenate governance_indicators_df into master_data
master_data_2 = concat_country_specific_data(master_data_1, economic_fitness_df, countries)

# Verify the combined data
print(master_data_2)

       Economy                                          Indicator       2000  \
0    Indonesia                   Economic Fitness Metric (Legacy)   1.548700   
1    Indonesia  Economic Fitness Ranking (1 = high, 149 = low)...  31.000000   
2    Indonesia                    Control of Corruption: Estimate  -0.908694   
3    Indonesia           Control of Corruption: Number of Sources   7.000000   
4    Indonesia             Control of Corruption: Percentile Rank  21.808510   
..         ...                                                ...        ...   
151      India  Direct Economic Loss Attributed to Disasters (...        NaN   
152      India  Proportion of Urban Population Living in Slums...  55.294770   
153      India  Coverage of Protected Areas in Relation to Mar...        NaN   
154      India  Protected Marine Areas (Exclusive Economic Zon...        NaN   
155      India             Number of Persons Affected by Disaster        NaN   

        2001       2002       2003     

### World Development Indicators

In [20]:
world_development_indicators_df = pd.read_csv('/content/drive/My Drive/IBM Mangrove Project/Data/India and Indonesia/World Development Indicators India Indonesia.csv')
print(world_development_indicators_df.shape)
world_development_indicators_df.head()

(115, 54)


,Country Name,Country Code,Series Name,Series Code,1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Indonesia,IDN,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,128.194,125.7,120.747,119.064,118.666,119.108,...,43.219,40.797,38.928,36.615,36.012,35.293,34.867,34.485,33.929,..
1,Indonesia,IDN,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,..,..,..,..,..,..,...,13.3566991621949,13.3367549992146,13.4926435578886,13.4787485470271,13.1566308462661,12.8084975586902,12.7126026577522,13.6984142796138,13.2802165425549,12.3996592154305
2,Indonesia,IDN,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,..,..,..,..,..,..,...,10.012833135186,10.3514328528261,10.6900325704661,11.0286322881062,11.0286322881062,11.0286322881062,11.0286322881062,11.0286322881062,..,..
3,Indonesia,IDN,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,..,..,..,..,..,..,...,87.4,..,..,92.6,90.9,93.6,94.7,..,..,..
4,Indonesia,IDN,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,..,..,..,..,..,..,...,1.77040203245853,1.89142782676209,1.88756448144649,1.84830385185678,1.94857443086369,2.12683683837158,2.24528577070833,2.07165896221472,..,..


In [21]:
# Rename necessary columns to match the desired structure
world_development_indicators_df.rename(columns={'Country Name': 'Economy', 'Series Name': 'Indicator'}, inplace=True)

# Drop columns that are not needed
world_development_indicators_df.drop(['Country Code', 'Series Code'], axis=1, inplace=True)

# Convert all column names to strings
world_development_indicators_df.columns = world_development_indicators_df.columns.map(str)

# Initialize a new dictionary to hold the updated column names
new_columns = {}

# Iterate over each column, extract the year, and update the column name if it's a year
for col in world_development_indicators_df.columns:
    match = re.search(r'(\d{4})', col)
    if match:
        year = match.group(1)
        new_columns[col] = year
    else:
        new_columns[col] = col

# Update the column names with the new names
world_development_indicators_df.rename(columns=new_columns, inplace=True)

# Focus on the year range 2000 to 2022, as per master_data structure
desired_years = [str(year) for year in range(2000, 2023)]

# Drop columns that are not within the desired year range
world_development_indicators_df = world_development_indicators_df[['Economy', 'Indicator'] + desired_years]

# Convert year column data to float64 and handle missing values
for year in desired_years:
    if year in world_development_indicators_df.columns:
        world_development_indicators_df[year] = pd.to_numeric(world_development_indicators_df[year], errors='coerce')
    else:
        # Add the missing year columns and fill with NaN if they don't exist
        world_development_indicators_df[year] = np.nan

# Reorder the columns to match the desired order
world_development_indicators_df = world_development_indicators_df[['Economy', 'Indicator'] + desired_years]

# Remove rows where all columns except 'Economy' and 'Indicator' are NaN
world_development_indicators_df.dropna(how='all', subset=desired_years, inplace=True)

# The DataFrame is now transformed to match the structure of master_data for the years 2000 to 2022.
# You can display the head to verify the structure.
print(world_development_indicators_df.head())
print(world_development_indicators_df.dtypes)

     Economy                                          Indicator       2000  \
0  Indonesia  Adolescent fertility rate (births per 1,000 wo...  54.557000   
1  Indonesia  Agriculture, forestry, and fishing, value adde...  15.678704   
2  Indonesia  Annual freshwater withdrawals, total (% of int...   5.611037   
3  Indonesia  Births attended by skilled health staff (% of ...  66.900000   
4  Indonesia             CO2 emissions (metric tons per capita)   1.310938   

        2001       2002       2003       2004       2005       2006  \
0  54.091000  52.425000  51.032000  48.246000  47.849000  47.363000   
1  15.994918  16.319669  15.185348  14.335780  13.126619  12.973803   
2   5.949637   6.288236   6.626836   6.965436   7.304035   7.642635   
3        NaN  68.400000  66.200000  71.500000        NaN        NaN   
4   1.391238   1.388515   1.496673   1.510302   1.495369   1.572351   

        2007  ...       2013       2014       2015       2016       2017  \
0  47.631000  ...  43.219000

In [22]:
# Countries for which the data is being added
countries = ['Indonesia', 'India']

# Use the function to concatenate governance_indicators_df into master_data
master_data_3 = concat_country_specific_data(master_data_2, world_development_indicators_df, countries)

# Verify the combined data
print(master_data_3)

       Economy                                          Indicator       2000  \
0    Indonesia  Adolescent fertility rate (births per 1,000 wo...  54.557000   
1    Indonesia  Agriculture, forestry, and fishing, value adde...  15.678704   
2    Indonesia  Annual freshwater withdrawals, total (% of int...   5.611037   
3    Indonesia  Births attended by skilled health staff (% of ...  66.900000   
4    Indonesia             CO2 emissions (metric tons per capita)   1.310938   
..         ...                                                ...        ...   
259      India  Direct Economic Loss Attributed to Disasters (...        NaN   
260      India  Proportion of Urban Population Living in Slums...  55.294770   
261      India  Coverage of Protected Areas in Relation to Mar...        NaN   
262      India  Protected Marine Areas (Exclusive Economic Zon...        NaN   
263      India             Number of Persons Affected by Disaster        NaN   

          2001       2002       2003   

### Agricultural Policy Monitoring and Evaluation

In [62]:
agricultural_policy_monitoring_and_evaluation_df = pd.read_csv('/content/drive/My Drive/IBM Mangrove Project/Data/India and Indonesia/Agricultural Policy Monitoring and Evaluation - India and Indonesia.csv')
agricultural_df_filtered = agricultural_policy_monitoring_and_evaluation_df[['Reference area', 'Measure', 'TIME_PERIOD', 'OBS_VALUE']]
print(agricultural_policy_monitoring_and_evaluation_df.shape)
agricultural_policy_monitoring_and_evaluation_df.head()

(2179, 15)


,Reference area,MEASURE,Measure,COMMODITY,Commodity,UNIT_MEASURE,Unit of measure,TIME_PERIOD,Time period,OBS_VALUE,Observation value,DECIMALS,Decimals,UNIT_MULT,Unit multiplier
0,Indonesia,PC,Payments based on current area planted / anima...,CPC_EX_TO,Total indicators,USD,US dollar,1995,NaN,2.3209,NaN,2,Two,6.0,Millions
1,Indonesia,PC,Payments based on current area planted / anima...,CPC_EX_TO,Total indicators,USD,US dollar,1996,NaN,3.0124,NaN,2,Two,6.0,Millions
2,Indonesia,PC,Payments based on current area planted / anima...,CPC_EX_TO,Total indicators,USD,US dollar,1997,NaN,2.6960,NaN,2,Two,6.0,Millions
3,Indonesia,PC,Payments based on current area planted / anima...,CPC_EX_TO,Total indicators,USD,US dollar,1998,NaN,1.4252,NaN,2,Two,6.0,Millions
4,Indonesia,PC,Payments based on current area planted / anima...,CPC_EX_TO,Total indicators,USD,US dollar,1999,NaN,2.1880,NaN,2,Two,6.0,Millions


In [63]:
# Adjusting the pivot table creation
agricultural_df_pivoted_correctly = agricultural_policy_monitoring_and_evaluation_df.pivot_table(
    values='OBS_VALUE',
    index=['Reference area', 'Measure'],
    columns='TIME_PERIOD',
    aggfunc='first'  # Using 'first' assuming there's at most one observation per year for each combination
).reset_index()

agricultural_df_pivoted_correctly

TIME_PERIOD,Reference area,Measure,1986,1987,1988,1989,1990,1991,1992,1993,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,India,Agricultural employment,NaN,NaN,NaN,NaN,NaN,63.5,63.1,62.7,...,46.4000,45.8000,45.2000,44.5000,43.9000,43.3000,41.4000,44.3000,44.0000,NaN
1,India,Agricultural knowledge and innovation system,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1303.0197,1147.1478,975.9203,990.2262,1332.2242,1324.2395,1346.0271,1096.0847,1273.9724,1280.8742
2,India,Agricultural knowledge generation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,795.6114,793.5814,533.9071,558.8858,947.6950,967.7616,930.0011,868.9588,934.4095,901.8398
3,India,Agricultural knowledge transfer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,507.4083,353.5664,442.0132,431.3403,384.5292,356.4779,416.0259,227.1259,339.5629,379.0345
4,India,Agricultural product safety and inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,202.6352,241.1738,141.1029,84.0078,99.4324,108.3264,58.3398,63.1500,49.1384,184.8032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Indonesia,Transfers to consumers from taxpayers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1943.5584,1592.5837,1695.5748,3735.2676,3261.6909,1349.1990,1431.7911,1415.1878,3153.5210,1931.6962
91,Indonesia,Transfers to producers from consumers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-24858.2966,-27352.8677,-23509.9430,-23096.3269,-22042.1410,-18738.2753,-17566.0697,-13633.5327,-8355.6477,-15840.8424
92,Indonesia,"Value of consumption, at farm gate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100668.7980,92524.5329,72255.2828,72597.3743,70766.1615,70829.3643,69657.8771,73139.9371,76085.9639,79013.1634
93,Indonesia,"Value of production, at farm gate",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111081.6300,110696.7257,104666.4681,111666.7986,115921.2829,104043.6032,101570.4980,106905.1170,120654.3062,115147.9480


In [64]:
desired_years = list(range(2000, 2023))
agricultural_df_pivoted_correctly = agricultural_df_pivoted_correctly[['Reference area', 'Measure'] + desired_years]
print(agricultural_df_pivoted_correctly.head())

TIME_PERIOD Reference area                                       Measure  \
0                    India                       Agricultural employment   
1                    India  Agricultural knowledge and innovation system   
2                    India             Agricultural knowledge generation   
3                    India               Agricultural knowledge transfer   
4                    India    Agricultural product safety and inspection   

TIME_PERIOD      2000      2001      2002      2003      2004      2005  \
0             59.6000   59.1000   58.4000   57.7000   56.8000   56.0000   
1            382.9248  418.9118  403.8927  448.0224  557.5987  644.5400   
2            290.7892  292.4228  304.8740  336.0521  387.1760  433.3360   
3             92.1356  126.4890   99.0187  111.9702  170.4227  211.2040   
4              5.1024    9.4481    8.7880   14.7155   25.7837   27.7773   

TIME_PERIOD      2006       2007  ...       2013       2014      2015  \
0             55.10

In [65]:
# Display the column names and data types of the pivoted DataFrame
print(agricultural_df_pivoted_correctly.dtypes)

TIME_PERIOD
Reference area     object
Measure            object
2000              float64
2001              float64
2002              float64
2003              float64
2004              float64
2005              float64
2006              float64
2007              float64
2008              float64
2009              float64
2010              float64
2011              float64
2012              float64
2013              float64
2014              float64
2015              float64
2016              float64
2017              float64
2018              float64
2019              float64
2020              float64
2021              float64
2022              float64
dtype: object


In [66]:
# Rename columns
agricultural_df_pivoted_correctly.rename(columns={'Reference area': 'Economy', 'Measure': 'Indicator'}, inplace=True)

# Convert year column names from integers to strings
agricultural_df_pivoted_correctly.columns = agricultural_df_pivoted_correctly.columns.map(lambda x: str(x) if isinstance(x, int) else x)

# Add missing year columns from master_data_3 to agricultural_df_pivoted_correctly with NaN values
for year_col in master_data_3.columns[2:]:  # Skip the first two columns which are 'Economy' and 'Indicator'
    if year_col not in agricultural_df_pivoted_correctly.columns:
        agricultural_df_pivoted_correctly[year_col] = np.nan

# Reset the index to convert all index columns back into regular columns
agricultural_df_pivoted_correctly.reset_index(inplace=True)

# If the 'TIME_PERIOD' column still exists and is not needed, you can drop it
if 'TIME_PERIOD' in agricultural_df_pivoted_correctly.columns:
    agricultural_df_pivoted_correctly.drop('TIME_PERIOD', axis=1, inplace=True)

# Ensure the columns are in the same order as in master_data_3
agricultural_df_pivoted_correctly = agricultural_df_pivoted_correctly[['Economy', 'Indicator'] + [str(year) for year in range(2000, 2023)]]
agricultural_df_pivoted_correctly

<ipython-input-66-050909936572>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agricultural_df_pivoted_correctly.rename(columns={'Reference area': 'Economy', 'Measure': 'Indicator'}, inplace=True)


TIME_PERIOD,Economy,Indicator,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,India,Agricultural employment,59.6000,59.1000,58.4000,57.7000,56.8000,56.0000,55.1000,54.2000,...,46.4000,45.8000,45.2000,44.5000,43.9000,43.3000,41.4000,44.3000,44.0000,NaN
1,India,Agricultural knowledge and innovation system,382.9248,418.9118,403.8927,448.0224,557.5987,644.5400,771.3226,1012.8224,...,1303.0197,1147.1478,975.9203,990.2262,1332.2242,1324.2395,1346.0271,1096.0847,1273.9724,1280.8742
2,India,Agricultural knowledge generation,290.7892,292.4228,304.8740,336.0521,387.1760,433.3360,503.1989,593.6931,...,795.6114,793.5814,533.9071,558.8858,947.6950,967.7616,930.0011,868.9588,934.4095,901.8398
3,India,Agricultural knowledge transfer,92.1356,126.4890,99.0187,111.9702,170.4227,211.2040,268.1236,419.1293,...,507.4083,353.5664,442.0132,431.3403,384.5292,356.4779,416.0259,227.1259,339.5629,379.0345
4,India,Agricultural product safety and inspection,5.1024,9.4481,8.7880,14.7155,25.7837,27.7773,30.6788,73.0876,...,202.6352,241.1738,141.1029,84.0078,99.4324,108.3264,58.3398,63.1500,49.1384,184.8032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Indonesia,Transfers to consumers from taxpayers,263.6121,237.5311,483.6928,547.8249,540.8712,655.2636,580.5526,720.4296,...,1943.5584,1592.5837,1695.5748,3735.2676,3261.6909,1349.1990,1431.7911,1415.1878,3153.5210,1931.6962
91,Indonesia,Transfers to producers from consumers,-1532.7343,-915.2190,-3653.0920,-4026.6294,-3278.6268,-1256.6392,-7820.2694,-9495.5910,...,-24858.2966,-27352.8677,-23509.9430,-23096.3269,-22042.1410,-18738.2753,-17566.0697,-13633.5327,-8355.6477,-15840.8424
92,Indonesia,"Value of consumption, at farm gate",22183.9398,20284.7119,25582.0253,30031.5414,30512.6506,31983.6754,41566.6964,51522.3422,...,100668.7980,92524.5329,72255.2828,72597.3743,70766.1615,70829.3643,69657.8771,73139.9371,76085.9639,79013.1634
93,Indonesia,"Value of production, at farm gate",22973.7545,21210.3649,27256.2990,33377.3681,35976.9245,37527.2757,50024.7308,61505.2144,...,111081.6300,110696.7257,104666.4681,111666.7986,115921.2829,104043.6032,101570.4980,106905.1170,120654.3062,115147.9480


In [68]:
# Countries for which the data is being added
countries = ['Indonesia', 'India']

# Use the function to concatenate governance_indicators_df into master_data
master_data_4 = concat_country_specific_data(master_data_3, agricultural_df_pivoted_correctly, countries)

print(master_data_4)

       Economy                                          Indicator      2000  \
0    Indonesia                            Agricultural employment  45.30000   
1    Indonesia       Agricultural knowledge and innovation system  42.54750   
2    Indonesia                  Agricultural knowledge generation  18.74730   
3    Indonesia                    Agricultural knowledge transfer  23.80020   
4    Indonesia              Based on farm receipts or farm income  10.37020   
..         ...                                                ...       ...   
354      India  Direct Economic Loss Attributed to Disasters (...       NaN   
355      India  Proportion of Urban Population Living in Slums...  55.29477   
356      India  Coverage of Protected Areas in Relation to Mar...       NaN   
357      India  Protected Marine Areas (Exclusive Economic Zon...       NaN   
358      India             Number of Persons Affected by Disaster       NaN   

        2001      2002     2003      2004     2005 

In [70]:
master_data_4.to_csv('/content/drive/My Drive/IBM Mangrove Project/master_data_w_added_variables.csv', index=False)